<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Read-in-Data" data-toc-modified-id="Read-in-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Read in Data</a></span><ul class="toc-item"><li><span><a href="#Read-in-excel-files-combining-ticker-symbols-with-the-IQID" data-toc-modified-id="Read-in-excel-files-combining-ticker-symbols-with-the-IQID-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Read in excel files combining ticker symbols with the IQID</a></span></li><li><span><a href="#Read-in-independent-variables,-join-tickers" data-toc-modified-id="Read-in-independent-variables,-join-tickers-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Read in independent variables, join tickers</a></span></li><li><span><a href="#Join-in-the-credit-rating-data" data-toc-modified-id="Join-in-the-credit-rating-data-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Join in the credit rating data</a></span></li></ul></li><li><span><a href="#Generate-Model-Data" data-toc-modified-id="Generate-Model-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Generate Model Data</a></span></li><li><span><a href="#Generate-Models" data-toc-modified-id="Generate-Models-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Generate Models</a></span><ul class="toc-item"><li><span><a href="#Support-Vector-Machines" data-toc-modified-id="Support-Vector-Machines-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Support Vector Machines</a></span></li><li><span><a href="#Random-Forest-Classifier" data-toc-modified-id="Random-Forest-Classifier-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Random Forest Classifier</a></span></li></ul></li><li><span><a href="#Prediction-Function" data-toc-modified-id="Prediction-Function-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Prediction Function</a></span></li><li><span><a href="#Plot-Accuracy" data-toc-modified-id="Plot-Accuracy-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Plot Accuracy</a></span></li><li><span><a href="#See-values" data-toc-modified-id="See-values-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>See values</a></span></li></ul></div>

In [1]:
import pandas as pd 
import numpy as np 
import plotly.graph_objects as go 
import seaborn as sns 
import matplotlib.pyplot as plt 
from tqdm import tqdm 

## Read in Data

---
### Read in excel files combining ticker symbols with the IQID 

In [2]:
ids= pd.DataFrame() 
## Read in ticker symbols
for i in range(1, 6): 
    df = pd.read_excel('capiq_data/in_process_ids/ids {}.xlsx'.format(i),
                       engine='openpyxl')[['ID', 'IQID', 'IQ Name']]
    ids = pd.concat([ids, df]) 

## See if there are any duplicates 
print(ids.duplicated().sum()) 
## See if there are any nulls 
print(ids.isna().sum()) 
ids.head() 

0
ID         0
IQID       0
IQ Name    0
dtype: int64


,ID,IQID,IQ Name
0,MMM,IQ289194,3M Company
1,ABT,IQ247483,Abbott Laboratories
2,ABBV,IQ141885706,AbbVie Inc.
3,ABMD,IQ247589,"Abiomed, Inc."
4,ACN,IQ972190,Accenture plc


### Read in independent variables, join tickers 

In [3]:
## Join IQID for inds so we can see the Ticker and the name 

ind_df = pd.read_csv('small_df.csv') 

ind_df = ind_df.merge(ids, on = ['IQID', 'IQ Name']) 
ind_df.drop(['Unnamed: 0', 'IQ Name', 'IQID', 
            'quarter'], axis = 1, inplace = True) 

## The data is quarterly, but we need to lok at it annually. 
## This means we have to take the mean of the data for all the quarters 
ind_df = ind_df.groupby(['year', 'ID']).mean().reset_index()
                      
ind_df.head() 

,year,ID,IQ_TOTAL_REV,IQ_NI_CF,IQ_AR,IQ_GP,IQ_TOTAL_ASSETS,IQ_AP,IQ_TOTAL_LIAB,IQ_TOTAL_DEBT,IQ_CASH_FINAN,IQ_TOTAL_EQUITY,IQ_CASH_EQUIV
0,2000,PYPL,4.262073,-44.73064,0.0,-11.11638,77.265667,2.933000,34.536333,0.076667,25.746083,42.729333,36.093333
1,2001,PYPL,26.078250,-26.95056,0.0,2.38625,198.586997,5.876978,126.882032,1.314882,3.115630,103.751965,101.747015
2,2002,PYPL,51.266500,0.88000,0.0,16.28150,265.305500,10.945000,107.277000,5.615000,34.556500,158.028500,89.832500
3,2006,HLT,2219.500000,162.00000,705.0,667.00000,16737.000000,1112.000000,13195.500000,7901.000000,-771.500000,3541.500000,126.000000
4,2007,HLT,1974.500000,130.00000,690.0,593.50000,16127.500000,1745.000000,12126.500000,6869.500000,-666.000000,4001.000000,149.000000


### Join in the credit rating data

We want an inner join so we only keep the companies that we have the credit rating for

In [4]:
## Join in the credit Ratings Data
credit_ratings = pd.read_csv('credit.csv')[['Year', 'TickerSymbol',
                                            'DomesticLTICRSPMthlyAvg']]
credit_ratings.rename({'DomesticLTICRSPMthlyAvg': 'rating'}, inplace = True, axis = 1)  

## Add credit ratings to df  
tot_df = credit_ratings.merge(ind_df, how = 'inner', 
                             left_on = ['Year', 'TickerSymbol'], 
                             right_on = ['year', 'ID'])
tot_df.drop(['Year', 'TickerSymbol'], axis = 1, inplace = True) 

tot_df.head() 

,rating,year,ID,IQ_TOTAL_REV,IQ_NI_CF,IQ_AR,IQ_GP,IQ_TOTAL_ASSETS,IQ_AP,IQ_TOTAL_LIAB,IQ_TOTAL_DEBT,IQ_CASH_FINAN,IQ_TOTAL_EQUITY,IQ_CASH_EQUIV
0,18.000000,2010,AAL,5586.00,-98.00,738.00,1286.00,25088.00,1156.00,29033.00,11136.0,349.00,-3945.00,168.00
1,19.500000,2011,AAL,5994.75,-494.75,923.50,1181.00,25366.75,1301.00,30467.25,11580.5,181.75,-5100.50,297.75
2,27.000000,2012,AAL,6213.75,-469.00,1117.25,1374.25,24168.50,1697.75,32422.50,10375.0,120.75,-8254.00,433.50
3,26.166667,2013,AAL,6678.00,-458.50,1379.75,1729.50,29781.50,1703.25,36592.75,13172.5,949.75,-6811.25,766.75
4,16.833333,2014,AAL,10662.25,720.50,1930.25,3009.50,43986.50,1543.50,40943.25,17203.0,-78.75,3043.25,1160.25


In [5]:
tot_df_clean = pd.DataFrame() 



for ticker in tqdm(tot_df['ID'].unique()): 
    
    
    ## small df is all the rows with the ticker, sort by year 
    small_df = tot_df[tot_df['ID'] == ticker].sort_values(by = 'year',
                                                             ascending = True)
    
    
    ## Insert a lead rating column. This is the predictor column, 
    ## as we are trying to predict the credit rating for the next year
    small_df.insert(loc = 0, column = 'lead_rating', 
               value = small_df.rating.shift(1)) 
    
    
    ## Set the index as the year and the ticker. 
    small_df.set_index(['year', 'ID'], inplace = True) 
    
    ## Take the difference between rows. We are looking to find 
    ## differences in credit rating, so we are going to compare it to 
    ## differences in dependent variables. 
    ## We can then drop the nulls. 
    small_df = small_df.diff().dropna()  
    
    tot_df_clean = pd.concat([tot_df_clean, small_df], axis = 0) 

tot_df_clean.head() 

100%|██████████| 306/306 [00:01<00:00, 223.54it/s]


,,lead_rating,rating,IQ_TOTAL_REV,IQ_NI_CF,IQ_AR,IQ_GP,IQ_TOTAL_ASSETS,IQ_AP,IQ_TOTAL_LIAB,IQ_TOTAL_DEBT,IQ_CASH_FINAN,IQ_TOTAL_EQUITY,IQ_CASH_EQUIV
year,ID,,,,,,,,,,,,,
2012,AAL,1.500000,7.500000,219.00000,25.75000,193.75000,193.25000,-1198.25000,396.7500,1955.2500,-1205.50000,-61.00000,-3153.50000,135.75000
2013,AAL,7.500000,-0.833333,464.25000,10.50000,262.50000,355.25000,5613.00000,5.5000,4170.2500,2797.50000,829.00000,1442.75000,333.25000
2014,AAL,-0.833333,-9.333333,3984.25000,1179.00000,550.50000,1280.00000,14205.00000,-159.7500,4350.5000,4030.50000,-1028.50000,9854.50000,393.50000
2012,PNW,-0.583333,-0.583333,15.10625,10.51725,-28.39100,23.65400,549.96775,-15.3985,382.2985,-186.43050,28.66800,167.66925,-166.78575
2013,PNW,-0.583333,-0.916667,38.20600,6.13300,30.82575,2.98675,239.18900,-0.1955,-19.5145,-71.94575,36.23175,258.70350,15.06900


## Generate Model Data

In [6]:
lead_rating = tot_df_clean['lead_rating'].to_numpy()


lead_rating[lead_rating == 0] = 0
lead_rating[(lead_rating > 0) & (lead_rating <= 1)] = 1 
lead_rating[(lead_rating < 0) & (lead_rating >= -1)] = -1 
lead_rating[lead_rating > 1] = 2 
lead_rating[lead_rating < -1] = -2
tot_df_clean['rating'] = tot_df_clean['lead_rating']
tot_df_clean['lead_rating'] = lead_rating

full_df = tot_df_clean 

In [7]:
tot_df_clean.lead_rating.value_counts().to_frame().style.bar()

,lead_rating
0.0,499
-1.0,148
1.0,71
-2.0,22
2.0,11


In [8]:
from sklearn.utils import resample 

## Resample the data

rating_0 = tot_df_clean[tot_df_clean.lead_rating ==0 ]
tot_df_clean_sampled = tot_df_clean[tot_df_clean.lead_rating !=0 ]
rating_0 = resample(rating_0, 
                   replace = True, 
                   n_samples = 150,
                    random_state = 123) 

tot_df_clean = pd.concat([rating_0, 
                          tot_df_clean_sampled], axis = 0) 

In [9]:
from sklearn.model_selection import train_test_split 

## Split into x and y
x = tot_df_clean.drop(['lead_rating'], axis = 1) 

y_numeric = tot_df_clean['rating'] 
y = tot_df_clean['lead_rating'] 


## Normalize the data, but we don't need to normalize the 
## dependent variable
x = (x - x.mean()) / (x.std())


## We'll set aside 10% of the data for testing
train_x, test_x, train_y, test_y = train_test_split(x, 
                                                    y, 
                                                    train_size = 0.9, 
                                                    random_state = 5)

train_x_num, test_x_num, train_y_num, test_y_num = train_test_split(x, y_numeric, train_size = 0.9, random_state = 5) 

## Generate Models 
### Support Vector Machines

In [10]:
from sklearn.model_selection import RepeatedStratifiedKFold 
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


def SVM_Fit(train_x, train_y, kernel,
            params = [10**x for x in np.arange(-1,3,0.9)]): 
    '''Fit the SVM Machine given the kernel type, parameters, 
    data''' 
    
    if kernel == 'linear': 
        parameters = {'C': params} 
    else: 
        parameters = {'C': params, 
                     'gamma': params} 
    
    cv = RepeatedStratifiedKFold(n_splits = 5, 
                                n_repeats = 5) 
    
    model = GridSearchCV(estimator = SVC(kernel = kernel), 
                        param_grid = parameters, 
                        cv = 2, 
                        verbose = 1) 
    
    model.fit(x, y) 
    return model

### Random Forest Classifier

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestClassifier(random_state = 200) 
rf.get_params()


# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


rf_random = RandomizedSearchCV(estimator = rf, 
                              param_distributions = random_grid, 
                              n_iter = 100, cv = 5, verbose = 2, 
                              random_state = 200, n_jobs = -1)


## Prediction Function

In [12]:
def Predict(fitted_model, test_x, test_y, name):
    prediction = fitted_model.predict(test_x) 
    score = accuracy_score(prediction, test_y) 
    prediction = pd.DataFrame({'prediction_{}'.format(name): prediction})
    print('The {} Model Score is: {}'.format(name, score)) 
    return prediction, score

In [13]:
## Make predictions
sigmoid = SVM_Fit(train_x, train_y, 'sigmoid') 
rbf = SVM_Fit(train_x, train_y, 'rbf') 
linear = SVM_Fit(train_x, train_y,'linear')
#poly = SVM_Fit(train_x, train_y, 'poly') 
poly = SVC(kernel = 'poly').fit(train_x, train_y)

rf_random.fit(train_x, train_y)

Fitting 2 folds for each of 25 candidates, totalling 50 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Fitting 2 folds for each of 25 candidates, totalling 50 fits
Fitting 2 folds for each of 5 candidates, totalling 10 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   57.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 500 out of 5

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=200),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=200, verbose=2)

In [14]:
sigmoid_predict, sigmoid_score = Predict(sigmoid, test_x, test_y, 'sigmoid') 
lin_predict, lin_score = Predict(linear, test_x, test_y, 'linear') 
poly_predict, poly_score = Predict(poly, test_x, test_y, 'poly') 
rbf_predict, rbf_score = Predict(rbf, test_x, test_y, 'radial') 


random_predict, random_score = Predict(rf_random, test_x, test_y, 
                                      'Random Forest')

The sigmoid Model Score is: 0.926829268292683
The linear Model Score is: 1.0
The poly Model Score is: 0.2682926829268293
The radial Model Score is: 1.0
The Random Forest Model Score is: 0.9512195121951219


## Plot Accuracy

In [15]:
fig = go.Figure() 
model_names = ['Sigmoid SVC', 'Radial SVC', 'Linear SVC', 'Polynomial SVC',
 'Random Forests']

model_accuracy = [sigmoid_score, rbf_score, lin_score, poly_score, random_score]

fig.add_trace(go.Bar(x = model_names, 
                    y = model_accuracy, 
                    text = model_accuracy, 
                    textposition = 'auto'))
fig.update_layout(title = 'Model Accuracy Scores')

fig.update_yaxes(title_text = 'Accuracy Score') 
fig.update_xaxes(title_text = "Model")
fig.show()

In [16]:
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.metrics import mean_squared_error
## OLS Linear Regression
ols = LinearRegression().fit(train_x_num, train_y_num)
ols_prediction = ols.predict(test_x_num)
ols_r2 = mean_squared_error( test_y_num,ols_prediction) 
ols_r2

1.5631035324251137e-31

In [17]:
from sklearn.metrics import mean_squared_error

def ContinuousPrediction(model, train_x, train_y, test_x, test_y): 

    params = [10**x for x in np.arange(-1,3,0.9)]

    model = GridSearchCV(estimator= model, param_grid = dict(alpha = params), cv = 5, verbose = 1)

    model.fit(train_x, train_y) 

    prediction = model.predict(test_x) 

    r_2 = mean_squared_error(test_y, prediction) 

    return model, prediction, r_2 

lasso, lasso_predict, lasso_r2 = ContinuousPrediction(Lasso(), train_x_num, train_y_num, test_x_num, test_y_num)
ridge, ridge_predict, ridge_r2 = ContinuousPrediction(Ridge(), train_x_num, train_y_num, test_x_num, test_y_num)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.1s finished


In [18]:
print('The Lasso Prediction R2 Score is: {}'.format(lasso_r2) ) 
print('The Ridge Prediction R2 Score is: {}'.format(ridge_r2) ) 
print('The OLS Prediction R2 Score is: {}'.format(ols_r2) ) 

The Lasso Prediction R2 Score is: 0.01226335677277638
The Ridge Prediction R2 Score is: 7.542438620456977e-08
The OLS Prediction R2 Score is: 1.5631035324251137e-31


In [19]:
from sklearn.svm import SVR


def SVM_Fit_Num(train_x, train_y, kernel,
            params = [10**x for x in np.arange(-1,3,0.9)]): 
    '''Fit the SVM Machine given the kernel type, parameters, 
    data''' 
    
    if kernel == 'linear': 
        parameters = {'C': params} 
    else: 
        parameters = {'C': params, 
                     'gamma': params} 
    
    cv = RepeatedStratifiedKFold(n_splits = 5, 
                                n_repeats = 5) 
    
    model = GridSearchCV(estimator = SVR(kernel = kernel), 
                        param_grid = parameters, 
                        cv = 2, 
                        verbose = 1) 
    
    model.fit(train_x, train_y) 

    return model

def PredictNum(fitted_model, test_x, test_y, name):
    prediction = fitted_model.predict(test_x) 
    score = mean_squared_error(test_y, prediction) 
    prediction = pd.DataFrame({'prediction_{}'.format(name): prediction})
    print('The {} Model Score is: {}'.format(name, score)) 
    return prediction, score

In [20]:
linear_num = SVM_Fit_Num(train_x_num, train_y_num, 'linear') 
#poly_num, poly_predict_num, poly_score_num = SVM_Fit_Num(train_x_num, train_y_num, test_x_num, test_y_num, 'poly') 
poly_num = SVR().fit(train_x_num, train_y_num) 
rbf_num = SVM_Fit_Num(train_x_num, train_y_num, 'rbf') 
sigmoid_num = SVM_Fit_Num(train_x_num, train_y_num,'sigmoid') 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Fitting 2 folds for each of 5 candidates, totalling 10 fits
Fitting 2 folds for each of 25 candidates, totalling 50 fits
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Fitting 2 folds for each of 25 candidates, totalling 50 fits
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.3s finished


In [21]:
sigmoid_predict_num, sigmoid_score_num = PredictNum(sigmoid_num, test_x_num, test_y_num, 'Sigmoid') 
rbf_predict_num, rbf_score_num = PredictNum(rbf_num, test_x_num, test_y_num, 'Radial') 
poly_predict_num, poly_score_num = PredictNum(poly_num, test_x_num, test_y_num, 'Polynomial')
lin_predict_num, lin_score_num = PredictNum(linear_num, test_x_num, test_y_num, 'Linear') 

The Sigmoid Model Score is: 0.19982459652072593
The Radial Model Score is: 0.10183665808343646
The Polynomial Model Score is: 0.09600518660786776
The Linear Model Score is: 0.0028633487564563307


In [22]:
fig = go.Figure() 
model_names = ['Sigmoid SVM', 'Radial SVM', 'Linear SVM', 'Polynomial SVM', 'OLS', 'Lasso', 'Ridge']

model_accuracy = [sigmoid_score_num, rbf_score_num, lin_score_num, poly_score_num, ols_r2, lasso_r2, ridge_r2]

fig.add_trace(go.Bar(x = model_names, 
                    y = model_accuracy, 
                    text = model_accuracy, 
                    textposition = 'auto'))
fig.update_layout(title = 'Model Accuracy Scores Numeric Prediction')

fig.update_yaxes(title_text = 'Mean Squared Error') 
fig.update_xaxes(title_text = "Model")
fig.show()

## See values

In [ ]:
import plotly.io as pio
pio.renderers.default = 'browser'

from cleaning_plots import * 
fig, metrics = PlotReg(full_df, 
        dep_cols = ['lead_rating', 'rating']).Plot()

In [ ]:
full_df.head() 

In [ ]:
fig.show() 